In [1]:
import pandas as pd
import numpy as np


In [2]:

df = pd.read_csv('jobOffers.csv', index_col=0, skipinitialspace=True)


In [3]:
#salary
df = df.drop(df[df['salaryRange'] == 'neplacená stáž'].index)
df['salLower'] = df['salaryRange'].str.split(" - ").str[0].str.split("  ").str[0]
df['salUpper'] = df['salaryRange'].str.split(" - ").str[1].str.split("  ").str[0]

df['salLower'] = df['salLower'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)
df['salUpper'].fillna(df['salLower'], inplace = True)
df['salUpper'] = df['salUpper'].replace({'k': '*1e3'}, regex=True).map(pd.eval).astype(int)

df['salAvg'] = (df['salUpper'] + df['salLower'])/2



In [4]:
#category and main programming language
df['category+progLang'] = df['category+progLang'].str.replace("Kategorie:","").str.strip()
df['category'] = df['category+progLang'].str.split(",").str[0]
df['mainProgLang'] = df['category+progLang'].str.split(",").str[1]


In [5]:
#seniority
df["seniority"] = df["seniority"].str.strip().replace({'Trainee, Junior': 'Junior', 'Trainee': 'Junior',
                                                          'Senior, Expert':'Senior',
                                                          'Expert':'Senior', 'Mid, Senior':'Mid Advanced'})


In [6]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")


In [7]:
#getting rid of the numbers, dots and converting to lowercase to standardise prog langugage
df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('\d+', '')
df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('.', '')
df["mainProgLang"]=df["mainProgLang"].str.lower()
#converting less demanded programming languages to 'other'
mask = df['mainProgLang'].map(df['mainProgLang'].value_counts())<16
df['mainProgLangAdj']=df['mainProgLang'].mask(mask,'other')
counts = df.groupby(['mainProgLangAdj']).size().sort_values(ascending=False)
print(counts)

mainProgLangAdj
other          201
 java          121
 python         83
 javascript     74
 net            59
 react          57
 php            48
 android        19
 node           19
dtype: int64


<ipython-input-7-5ea043900ff0>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('\d+', '')
<ipython-input-7-5ea043900ff0>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.loc[:,"mainProgLang"] = df.loc[:,"mainProgLang"].str.replace('.', '')


In [8]:
#adjusting remote with location -- Where is Na dálku in remote => location: Remote, 

df.loc[df.remote != "Na dálku ","remote"]= "Hybrid or no remote"
df['remoteAdj'] = ['Remote' if x == 'Na dálku ' else 'Hybrid or no remote' for x in df['remote']]



In [9]:
#Work language
df["workLanguage"]=df["workLanguage"].str.replace("Work language:","")
df['language'] = df['workLanguage'].str.split(',')

df.groupby(['workLanguage']).size()
langList = ['angličtina','česky']
for lang in langList:
    df[lang] = df['workLanguage'].str.contains(lang)

#['angličtina', 'česky' ]

In [10]:
#Because we are interested only on the job offers relevant for the Czech people, we are removing all jobs where neither Czech nor English is used as a speaking language.
df = df[(df['angličtina'] == True)| (df['česky'] == True)]

In [11]:
#do we keep bonus? (and jobTitles and company?)
df.groupby("Bonus").size()

#base on this we decide to not include bonus in the final dataset

Bonus
+1%           1
+10%         24
+12%          1
+13%          1
+15%          5
+2%           1
+20%          2
+30%          2
+5%           5
+50%          2
+7%           1
10000 PLN     1
20k CZK       1
dtype: int64

In [12]:
#controlling for missing values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 658 entries, 0 to 716
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   jobTitles          658 non-null    object 
 1   company            658 non-null    object 
 2   salaryRange        658 non-null    object 
 3   Bonus              47 non-null     object 
 4   category+progLang  658 non-null    object 
 5   seniority          658 non-null    object 
 6   workLanguage       658 non-null    object 
 7   location           548 non-null    object 
 8   remote             658 non-null    object 
 9   salLower           658 non-null    int64  
 10  salUpper           658 non-null    int64  
 11  salAvg             658 non-null    float64
 12  category           658 non-null    object 
 13  mainProgLang       625 non-null    object 
 14  mainProgLangAdj    625 non-null    object 
 15  remoteAdj          658 non-null    object 
 16  language           658 non

In [13]:
#subseting dataset
df_final=df[['jobTitles','seniority', 'workLanguage','angličtina', 'česky','location','remoteAdj','salAvg', 'category',
       'mainProgLangAdj']]
df_final.head()


,jobTitles,seniority,workLanguage,angličtina,česky,location,remoteAdj,salAvg,category,mainProgLangAdj
0,Java Software Developer,Junior,angličtina,True,False,NaN,Remote,29050.0,Backend,java
1,C++ Developer,Mid Advanced,česky,False,True,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Hybrid or no remote,105000.0,Backend,other
2,C# Developer,Mid Advanced,česky,False,True,"Prague, Altajská 1568/4 (Po pandemi)• Prague,...",Hybrid or no remote,105000.0,Backend,other
3,Java developer u Medvědů,Mid Advanced,česky,False,True,"Praha, Vítkova 241/10 • Praha, Vítkova 241/10",Hybrid or no remote,90000.0,Backend,java
4,Senior Backend Developer,Mid Advanced,"česky, angličtina",True,True,"Praha, Rohanské nábř. 19 • Remote• Praha, Roh...",Remote,80000.0,Backend,node


In [14]:
#making sure there are no whitespaces left
df_final.columns = df_final.columns.str.strip()



In [15]:
#to CSV
df_final.to_csv("cleaned_data.csv")